https://vinesmsuic.github.io/2020/09/29/robotics-purepersuit/#theoretical-derivation
https://github.com/arimb/PurePursuit
Implementation of an Adaptive Pure Purusit Controller

In [1]:
from pygame.locals import *
import pygame
import time
from pygame.math import Vector2
import math

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#odom code for real world
# x_loc = 0
# y_loc=0
# def getCurrLoc():
#     distance = (Change_in_left_encoder+change_in_right_encoder)/2.0
#     x_loc += distance *cos(gyro_angle)
#     y_loc += distance * sin(gyro_angle)

#lookahead distances ranging between 0 and 2r are all equally admissable.

In [23]:
with open("data.csv") as file:
    path = [([float(x) for x in line.split(",")]) for line in file.readlines()]

In [24]:
pos = (0,0)

In [25]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [26]:
def closest():
    global path, pos
    minDist = (0, distance(pos[0], pos[1],path[0][0], path[0][1]))
    for i in range(len(path)):
        dist = distance(path[i][0], path[i][1],pos[0], pos[1])
        if (dist<minDist[1]):
            minDist = (i,dist)
    return minDist[0]

In [27]:
closest()

0

In [34]:
l= 15
angle = 0 #radians

In [35]:
def lookahead():
    global pos, path, l, t_i
    for i, p in enumerate(reversed(path[:-1])):
        i_ = len(path)-2 - i
        d = (path[i_+1][0]-p[0], path[i_+1][1]-p[1])
        f = (p[0]-pos[0], p[1]-pos[1])
        a = sum(j**2 for j in d)
        b = 2*sum(j*k for j,k in zip(d,f))
        c = sum(j**2 for j in f) - l**2
        disc = b**2 - 4*a*c
        if (disc>=0):
            t1 = (-b+math.sqrt(disc))/(2*a)
            t2 =  (-b-math.sqrt(disc))/(2*a)
            if (0<=t1<=1):
                t_i=i_
                t = t1
                return p[0]+t*d[0], p[1]+t*d[1]
            if (0<=t2<=1):
                t_i=i_
                t = t2
                return p[0]+t*d[0], p[1]+t*d[1]
    t_i=0
    t=0
    return path[closest()][0:2]

In [36]:
lookahead()

-1093.0797775034503
-1562.2767575870068
-1865.1810166925002
-2048.4254132530436
-2155.895543494249
-2217.911006624015
-2253.3820619305575
-2273.5687406855586
-2285.0197584951047
-2291.494569145154
-2295.133489038881
-2297.1448599284813
-2298.1987296494462
-2298.6492643534966
-2298.652118399191
-2298.2184641170834
-2297.2218867220795
-2295.362314755544
-2292.0770143579975
-2286.373245780714
-2276.5353323492923
-2259.625929431928
-2230.6533742389192
-2181.2163855959907
-2097.3938561499667
-1956.7446731333987
-1724.9386554585542
-1355.0554112513184
-800.8338232478865
-80.34209013362283
347.4176963489733
585.7751441649655
721.0188401487071
798.1403992510618
842.1152085862389
867.1496257326253
881.3760730105549
889.4487633905462
894.0257031663218
896.6191153045633
898.0858074585547
898.9155221890326
899.3888434190536


(10.606376592073467, 10.606826838744757)

In [41]:
sign = lambda x:math.copysign(1, x)

In [42]:
def curvature(lookahead):
    global path, pos, angle, l 
    a = -math.tan(3.1415/2 - angle)
    b = 1
    c = math.tan(3.1415/2 - angle) * pos[0] - pos[1]
    x = abs(a*lookahead[0] + b*lookahead[1] + c)/math.sqrt(a**2+b**2) #point line dist
    side = sign(math.sin(3.1415/2 - angle) * (lookahead[0]-pos[0]) - math.cos(3.1415/2 - angle)*(lookahead[1]-pos[1]))
    #^ needed to figure out which side the robot is on
    curv = (2*x)/(l**2)
    curv*=side
    return curv
    

In [43]:
curvature(lookahead())

0.12379440766521925

In [44]:
def turn (curv, vel, width):
    return [vel*(2+curv*width)/2, vel*(2-curv*width)/2]

In [45]:
width = 10
dt = 0.005
maxVelChange = 500
wheels = [0,0]
lastwheels = [0,0]
pos=(0,0)
angle = math.atan2(path[1][0], path[1][1])
t_i = 0
while closest() != len(path)-1:
    look = lookahead()
    close = closest()
    curv = curvature(look) if t_i>close else 0.00001
    vel = path[close][2]
    last_wheels = [wheels[0], wheels[1]]
    wheels = turn(curv, vel, width)
    print(str(pos[0])+"\t"+str(pos[1]))
    for i, w in enumerate(wheels):
        wheels[i] = last_wheels[i] + min(maxVelChange*dt, max(-float(maxVelChange)*dt, w-last_wheels[i]))
    pos = (pos[0] + (wheels[0]+wheels[1])/2*dt * math.sin(angle), pos[1] + (wheels[0]+wheels[1])/2*dt * math.cos(angle))
    angle += math.atan((wheels[0]-wheels[1])/width*dt)
    

0	0
0.011605700030984814	0.0046430299149153795
0.03481710009295444	0.013929089744746138
0.06963420018590888	0.027858179489492277
0.11605700030984814	0.046430299149153795
0.1740855004647722	0.06964544872373069
0.24371970065068108	0.09750362821322296
0.3249596008675748	0.13000483761763063
0.41780520111545333	0.16714907693695366
0.5222565013943167	0.20893634617119208
0.6383135017041648	0.25536664532034586
0.7659762020449977	0.30643997438441506
0.9052446024168155	0.3621563333633996
1.0561187028196182	0.4225157222572995
1.2185985032534057	0.4875181410661148
1.3926840037181778	0.5571635897898455
1.5686024191302943	0.627542322926347
1.7528967575768695	0.7012719072880111
1.937191118301885	0.7750014359625006
2.1214855022075696	0.8487309066945534
2.305779910184656	0.9224603172576379
2.490074343118406	0.9961896654388884
2.6823802470096374	1.0731240270866733
2.874686179898775	1.1500583162509603
3.0669921426848332	1.2269925306844809
3.2592981362017044	1.3039266683027373
3.4592949232471204	1.3839374

55.46163676479794	27.925450340606748
55.78816511473035	28.26497671264204
56.11753720487946	28.609635411065387
56.44847193813024	28.95908382461139
56.78150242846936	29.315083292861903
57.11626522201592	29.67826856839782
57.45145617956757	30.048244670177084
57.78754444179797	30.42677158332818
58.12320041191034	30.813398187064635
58.45883989466479	31.209890354263504
58.79310410297568	31.61572569483898
59.11954557436766	32.02416913908184
59.44022846564277	32.437149241063736
59.75335069926751	32.8516906847086
60.06123041734053	33.270140467703634
60.362122573458386	33.689490066714306
60.65828681419472	34.11219199279548
60.948024592806256	34.535207612777306
61.2335383870317	34.96108560987312
61.51316655538142	35.38675953846998
61.789056782208	35.814865586971145
62.05957567566175	36.24231118641393
62.32681934158229	36.67181210052679
62.58917489197289	37.10025087196561
62.848690712018616	37.530415671593325
63.103767532224914	37.959165054640195
63.356410339667804	38.38935318734146
63.60502739674

13.085744157195698	99.91430330831749
12.87244528331391	99.91628835699692
12.659146413666877	99.91827386066014
12.445847548255568	99.92025981930715
12.240916565478106	99.92216830440321
12.03598558661232	99.92407720948457
11.831054611659072	99.92598653455119
11.626123640619223	99.92789627960308
11.421192673493634	99.92980644464022
11.224986139773604	99.93163569122527
11.028779609643209	99.93346532279708
10.832573083103204	99.93529533935565
10.636366560154345	99.93712574090097
10.440160040797387	99.93895652743304
10.25308438321209	99.94070248176435
10.06600872889352	99.94244878608392
9.878933077842328	99.94419544039174
9.691857430059171	99.9459424446878
9.504781785544704	99.9476897989721
9.317706144299581	99.94943750324464
9.140230512479002	99.95109585404725
8.962754883601999	99.9527545198396
8.785279257669131	99.9544135006217
8.607803634680959	99.95607279639353
8.430328014638041	99.95773240715509
8.26300228197571	99.9592974012486
8.095676551932367	99.96086267533337
7.928350824508479	99.9